# NERC FSF -- Online Workshop

## Stated Aims

This notebook will take you through the steps of visualising data from a field spectrometer and turning it into a spectral
library to use for image classification.  The data here is was collected on the 17th September 2020 at Dryden Farm. 
Spectra were recorded of the different end members in the field: soil "soiltrue", cut barley stubble "brashes", vegetation that was growing through the planted rows of barley as weeds "vegetation", and moss "mosspatches". In your file explorer, open the "Photos" folder to see a photo of each scene captured by the field spectrometer.

## Introduction to Python

Python is a computing language that is extensively used in the scientific community. While it may not be as effective for computationally demanding tasks as some other languages, such as C or Mathematica, its open source nature, large number of freely available and easy to install modules, and clear, easy to understand syntax, make it ideal for entry level (and beyond) scientific computing. 
In particular, **the Jupyter Notebook**, developed by Project Jupyter, is an excellent tool for both learning and demonstrating Python. Jupyter Notebooks are made up of **cells**, small snippets of code that can be run consecutively, allowing you to develop -- and to understand -- Python scripts. We'll be using it to teach you more about Python and its use in the processing of field spectroscopy data.

## Using on your own data

This workbook is designed such that you can paste your own data into the "data" folder and run through the processing with minimal modification. If you would like assistance in doing this in the future please email us at fsf@nerc.ac.uk


### First Steps -- Importing Modules

The first stage in any workflow in Python is to import the modules (which are collections of functions) that you will be using. In this analysis, we import a number of modules, the function of which is described in the comments (the light cyan text prefixed by the hash symbol). Note, modules must be installed first before use -- this can be handled by the **conda** package manager, or the **pip** package manager (please get in touch with us if you require help in setting this up). Here, the most important module to note is **SpecDAL**, a module designed to handle and process field spectroscopy data. To run the cell, **press Shift and ENTER**, or press **Run** in the ribbon above. 

In [ ]:
!pip install git+https://github.com/NERC-FSF/FieldSpecUtils.git -U
import os    #a module that allows us to "talk" with our operating system, neccessary for handling paths, files and so on
from pathlib import Path #a more specific path handling module that allows us to quickly create path names
from specdal import Collection, Spectrum, read #We import our main package, specdal, and ask to only import certain key functions
import pandas as pd   #Pandas is a powerful module for the handling of data sets
from matplotlib import pyplot as plt   #matplotlib emulates the functions of MATLAB. Here, we tell matplotlib to only import one of it's functions, the pyplot function, and to import it with the identifier 'plt'
from matplotlib.pyplot import ylabel, xlabel, title, legend   #We go even further by asking pyplot to only import some of it's subfunctions
from scipy.signal import savgol_filter   #We will use this to smooth our data
import numpy as np #We require numpy for integration functions that will be used during convolution
import shutil #A clean up utility to remove files at the end of our session 
import FieldSpecUtils #NERC FSF utilities for the derivation of vegetiation indices and convultion of hyperspectral data

### Paths
Knowing where your data is located is naturally critical for processing. For this tutorial, your data folder containing you raw .sig files (**Data**) has been bundled along with the Notebook.  
In the cell below, we set the **path** to the data folder, using the os module's inbuilt functions.
Check the printed output here to make sure that this is the correct path to your .sig files

In [ ]:
cwd = Path.cwd()
Home_Dir = cwd
Data_Dir = cwd / "Data" #The sub-directory where our .sig files are located
csv_Dir = str(cwd / "csv") #The sub-directory which will store generated .csv files which at the end of our workshop will be deleted 
os.mkdir(csv_Dir) #We create the sub directory using the path names generated above, starting with "csv_Dir"
print(Home_Dir)
print(Data_Dir)
print(csv_Dir)

### SpecDAL
We will now use the SpecDAL package to read our .sig files, and assign them to a collection of spectra which we can analyse and process further.

In [ ]:
DrydenEndMembers = Collection(name='DrydenEndMEmbers', directory ='Data')

Let's now take a look at this spectra collection. First of all, let's have a look at the header information for the
first 10 spectra in our collection...

In [ ]:
print(type(DrydenEndMembers.spectra))
for s in DrydenEndMembers.spectra[0:10]:
    print(s)

We can take a visual look at the data too...

In [ ]:
DrydenEndMembers.plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

### Grouping Data 
It can be difficult to assess your data by viewing all spectra at once. 
We will group the data based on the vegetation type using SpecDAL's groupby function. 
This groups data based on their file names, in this case the name before the "_" separator e.g. "grass".

In [ ]:
groups = DrydenEndMembers.groupby(separator='.', indices=[0])
group_names = list(groups.keys())
print(group_names)

We can now limit our graph to show only one vegetation type. Run this cell, then change **'Brashes'** to another one of the group names to plot that data.

In [ ]:
groups['Brashes'].plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

We can then average each of these groups to produce a collection of means, called "Species_means". We will be returning to this collection later on, but for now, let's continue to look at individual spectra...

In [ ]:
Species_means = Collection(name='Species_means')
for group_key, group_collection in groups.items():
    Species_means.append(group_collection.mean())

Species_means.plot(title='Group means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Spectral Reflectance")
plt.show()

Now that we've had a look at the data, we can do some preprocessing

**THE NEXT SECTION GOES THROUGH SOME PREPROCESSING STEPS**

### Interpolation

Notice that the steps between the wavelengths correspond to the spectral resolution of the instrument.
We want to interpolate reflectance measurements that correspond to wavelengths with 1.0 nm spacing.
This can be done using specdal as follows -- 

In [ ]:
DrydenEndMembers.interpolate(spacing=1, method='linear')
print(type(DrydenEndMembers.spectra))
for s in DrydenEndMembers.spectra[0:5]:
    print(s)

###  Relative vs Absolute Reflectance
Notice from our previous graphs that the y-axes are labelled "Relative Reflectance". This is because these spectra were recorded relative to the reflectance of the white Spectralon panel. We take measurements relative to this panel to approximate the total irradiance coming from the sun and hitting the object you are interested in taking a spectral measurement of. Because the panel does not reflect 100% of the light that hits it in a completely uniform manner we need to adjust our  "Relative Reflectance" measurements using the panel's known, laboratory calibrated reflectance to convert our field measurements to absolute reflectance.

We can derive the absolute reflectance then by multiplying our field measurements by the spectral reflectance of the panel for each wavelength. We can use this file to convert our measurements as follows. We are creating new dataframes that will host the corrected values, but the original dataframe can be corrected without having to assign a new identifier! 

In [ ]:
reference_panel = pd.read_csv((os.path.join(cwd, "SRT_44.csv")), index_col = "wavelength") 
AbsoluteRef_DrydenEM = DrydenEndMembers.data.mul(reference_panel['Reflectance'], axis = 0)

Let's compare a few of the relative reflectance spectra, in the dataframe **DrydenEndMembers**, with their corresponding, corrected spectra in **AbsoluteRef_DrydenEM**.  We can achieve this by calling specific columns in our uncorrected and corrected dataframes, concatenating them, and then comparing:

In [ ]:
Relative = DrydenEndMembers.data[['Brashes.0000_moc', 'MossPatches.0000_moc',
                                  'soiltrue.0000_moc', 'vegetation.0000_moc' ]].copy().add_suffix('_Relative')

Absolute = AbsoluteRef_DrydenEM[['Brashes.0000_moc', 'MossPatches.0000_moc',
                                 'soiltrue.0000_moc', 'vegetation.0000_moc' ]].copy().add_suffix('_Absolute')


Relative_vs_Absolute = pd.concat([Relative, Absolute], axis=1)

Relative_vs_Absolute.head()
Relative_vs_Absolute.plot(figsize=(12, 6), legend = True, ylim=(0.0,0.45), xlim=(350, 2500))
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
xlabel("Wavelength (nm)")
ylabel("Reflectance")
plt.show()


### Smoothing and Water Band Removal
You may also notice a few noisy regions in the spectra. Can you see them?
Let's zoom in around two regions in the Vegetation collection -- 1350 to 1460 nm, and 1790 to 1970 nm...

In [ ]:
AbsoluteRef_DrydenEM.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1300, 1500))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

AbsoluteRef_DrydenEM.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1750, 2000))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

These noisy regions are caused by the absorption of solar irradiance by water vapour in the atmopshere and can interfere with classification or use in indices. If the noise isn't too bad then smoothing may be suffcient to get rid of it, otherswise we will need to remove the noisy regions. Let's smooth our data set using a Savitsky-Golay filter.

In [ ]:
AbsoluteRef_DrydenEM.to_csv(csv_Dir + "/" + r'AntarcticVeg_AbsRef.csv')
AbsRef_Smoothing_DrydenEM = pd.read_csv(csv_Dir + "/" + r'AntarcticVeg_AbsRef.csv', index_col=0, header=0)
AbsRef_Smoothing_DrydenEM.drop(AbsRef_Smoothing_DrydenEM.index[0:9], inplace = True)
AbsRef_Smoothing_DrydenEM.drop(AbsRef_Smoothing_DrydenEM.index[2151:], inplace = True)
AbsRef_Smoothing_DrydenEM_smoothed = pd.DataFrame(savgol_filter(AbsRef_Smoothing_DrydenEM, 41, 2, axis=0),
                                columns=AbsRef_Smoothing_DrydenEM.columns,
                                index=AbsRef_Smoothing_DrydenEM.index)

Let's take a look at how that changed things --

In [ ]:
AbsRef_Smoothing_DrydenEM_smoothed.plot(figsize=(8, 5), legend = None, ylim=(-1,1), xlim=(1300, 1500))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

AbsRef_Smoothing_DrydenEM_smoothed.plot(figsize=(8, 5), legend = None, ylim=(-2,2), xlim=(1750, 2000))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
plt.show()

In this case, smoothing has managed to remove most of the noise from the c. 1400nm water absorption feature, but not the one at c. 1900nm. Lets completely remove the water band at c. 1900nm...

In [ ]:
AbsRef_Smoothing_DrydenEM_smoothed.to_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_WBRemoval.csv')
AbsRef_Smoothed_WBRemoved_DrydenEM = pd.read_csv(csv_Dir + "/" + 'DrydenEM_AbsRef_WBRemoval.csv', index_col=0, header=0)
#AbsRef_Smoothed_WBRemoved_DrydenEM.loc[1350:1460] = "" #this can be used if you need to remove noise @ 1400nm. 
AbsRef_Smoothed_WBRemoved_DrydenEM.loc[1790:1970] = ""
AbsRef_Smoothed_WBRemoved_DrydenEM.to_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_WBRemoval.csv')
AbsRef_Smoothed_WBRemoved_DrydenEM = pd.read_csv(csv_Dir + "/" + 'DrydenEM_AbsRef_WBRemoval.csv', index_col=0, header=0)

We now conduct the same operations on our averaged, grouped spectra. For simplicity, we will combine all operations in one cell:

In [ ]:
Species_means.interpolate(spacing=1, method='linear')
AbsoluteRef_DrydenEM_groups = Species_means.data.mul(reference_panel['Reflectance'], axis = 0)

AbsoluteRef_DrydenEM_groups.to_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_Groups.csv')
AbsRef_Smoothing_DrydenEM_groups = pd.read_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_Groups.csv', index_col=0, header=0)
AbsRef_Smoothing_DrydenEM_groups.drop(AbsRef_Smoothing_DrydenEM_groups.index[0:9], inplace = True)
AbsRef_Smoothing_DrydenEM_groups.drop(AbsRef_Smoothing_DrydenEM_groups.index[2151:], inplace = True)
AbsRef_Smoothing_DrydenEM_groups_smoothed = pd.DataFrame(savgol_filter(AbsRef_Smoothing_DrydenEM_groups, 41, 2, axis=0),
                                columns=AbsRef_Smoothing_DrydenEM_groups.columns,
                                index=AbsRef_Smoothing_DrydenEM_groups.index)


AbsRef_Smoothing_DrydenEM_groups_smoothed.to_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_WBRemoval_groups.csv')
AbsRef_Smoothed_WBRemoved_DrydenEM_groups = pd.read_csv(csv_Dir + "/" + 'DrydenEM_AbsRef_WBRemoval_groups.csv', index_col=0, header=0)
#AbsRef_Smoothed_WBRemoved_DrydenEM_groups.loc[1350:1460] = "" #this can be used if you need to remove noise @ 1400nm. 
AbsRef_Smoothed_WBRemoved_DrydenEM_groups.loc[1790:1970] = ""
AbsRef_Smoothed_WBRemoved_DrydenEM_groups.to_csv(csv_Dir + "/" + r'DrydenEM_AbsRef_WBRemoval_groups.csv')
AbsRef_Smoothed_WBRemoved_DrydenEM_groups = pd.read_csv(csv_Dir + "/" + 'DrydenEM_AbsRef_WBRemoval_groups.csv', index_col=0, header=0)


We can now export this collection to a .csv, which will act as our spectral library in further processing...

In [ ]:
AbsRef_Smoothed_WBRemoved_DrydenEM_groups = AbsRef_Smoothed_WBRemoved_DrydenEM_groups.dropna()

AbsRef_Smoothed_WBRemoved_DrydenEM_groups.interpolate(method='linear', axis=0).ffill().bfill()
AbsRef_Smoothed_WBRemoved_DrydenEM_groups.to_csv("Mean_plot_spectra.csv")
AbsRef_Smoothed_WBRemoved_DrydenEM_groups.to_csv(r'Dryden_Spectral_Lib_SNAP.csv', sep='\t', encoding='utf-8')

Let's look at how this spectral library looks:

In [ ]:
AbsRef_Smoothed_WBRemoved_DrydenEM_groups.plot(figsize=(9, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Absolute Reflectance")
title("Mean absolute reflectance for Dryden Farm end members")
legend(title = "Collection")
plt.show()

## Vegetation Indices
With your data now processed and converted into absolute spectral reflectance, we can look more closely at differences between vegetation types. We can use dimensionality reduction methods such as spectral indices 

https://fsf.nerc.ac.uk/resources/learning/HSI.shtml

A number of spectral indices have been designed to highlight different vegetation properties. In this next section, we will use some of them to explore differences between our vegetation types. For a full description of the indices used, please visit: https://www.l3harrisgeospatial.com/docs/NarrowbandGreenness.html

Firstly, let's import the **Vegetation_Indices** module from FSF's FieldSpecUtils module. 
**Vegetation_Indices** has a number of fucntions, which you can see by using **help(FieldSpecUtils.Vegetation_Indices)**. Each function has a description of the vegetation index it calculates. You can use the **help("module name")** feature for other modules to find out more about them.



In [ ]:
from FieldSpecUtils import Vegetation_Indices
help("FieldSpecUtils.Vegetation_Indices")

With this mind, let's calculate the Normalized Differential Vegetation Index for our individual and group spectra:

In [ ]:
# use a "#" to comment out individual or grouped data

#Vegetation_Indices.RENDVI(AbsRef_Smoothed_WBRemoved_DrydenEM)
Vegetation_Indices.RENDVI(AbsRef_Smoothed_WBRemoved_DrydenEM_groups)

### Convolution
If your research relates to specific multispectral imaging sensors e.g. Sentinel 2, it can be useful to resample 
your hyperspectral data so that it matches the bands of your specific sensor. 
We can do this by convolving the hyperspectral data to a multispectral sensor's "spectral response function". 
Read more on this here: HYPERELINK
    
We will be using this hyperspectral data to classify a Worldview 3 image 
https://earth.esa.int/eogateway/missions/worldview-3 
So lets convolve our data so that it matches Worldview 3.

To do this, we use another function in our FSF module called "Convolution". Let's import that:

In [ ]:
from FieldSpecUtils import Convolution
help("FieldSpecUtils.Convolution")

As with **Vegetation_Indices**, you can find out more about the functions included in **Convolution** by typing **help("FieldSpecUtils.Convolution")**. Notice the function **S2** and it's description. We will use **S2** to convolve our spectral library to Sentinel-2 bands. To do so, we import the spectral sesponse function for Sentinel-2, found in **Sentinel 2 SRF.csv**. It shows the relative response to specific wavelengths of light for each band of the imager.

https://fsf.nerc.ac.uk/resources/learning/SRF.shtml


In [ ]:
Bands = pd.read_csv("Sentinel 2 SRF.csv", index_col=0, header=0)

In [ ]:
Convolution.S2(AbsRef_Smoothed_WBRemoved_DrydenEM_groups, Bands)

Our hyperspectral data has now been convolved to multispectral bands equivalent to Sentinel-2. We can see how this looks by plotting the data output of the convolution, **Ploths_with_convolved_bands.csv**:

In [ ]:
collated_convolved = pd.read_csv("Plots_with_convolved_bands.csv", index_col = 0)
collated_convolved.plot(figsize=(9, 6), legend = True, ylim=(0,1), linestyle='--', marker='o')
xlabel("Centre Wavelength (nm) of Band with Band Number")
ylabel("Absolute Reflectance")
title("Hyperspectral convolved broadband reflectance")
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
           ["490 -- Band 2", "560 -- Band 3", "665 -- Band 4", "705 -- Band 5", "740 -- Band 6",
            "783 -- Band 7", "842 -- Band 8", "865 -- Band 8a", "1610 -- Band 11", "2190 -- Band 12"],
           rotation=20)
legend(title = "Plot number and vegetation type")
plt.tight_layout()
plt.savefig("Convolved_Hyperspectral_Plot_Data_to_S2_Multispectral_Bands.pdf")
plt.show()

## Calculating Vegetation Indices using convolved data
As our hyperpsectral data convolved to Sentinel-2 equivalent multispectral bands, we can also calculate a value for the Normalized Difference Vegetation Index using band values. For Sentinel-2 bands, NDVI can be calculated, according to https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndvi/, as:

$$
\begin{align}
NDVI = \dfrac{B_8 - B_4}{B_8 + B_4}
\end{align}
$$

Where ${B_8}$ is Band 8 (centre wavelength = 842 nm) and ${B_4}$ is Band 4 (centre wavelength = 665). The cell below runs this equation using your convolved hyperspectral data, and appends the NDVI as a new row to your dataframe **collated_convolved**:

In [ ]:
collated_convolved.loc['NDVI'] = ((collated_convolved.loc['Band 8 - 842'] - collated_convolved.loc['Band 4 - 665']) / 
                                  (collated_convolved.loc['Band 8 - 842'] + collated_convolved.loc['Band 4 - 665'])) 

print(collated_convolved.loc['NDVI'])

We will now create a new data frame that will compare the NDVI values generated from the hyperspectral data and the same hyperspectral data, but convolved to multispectral data. 

In [ ]:
Hyperspectral_NDVI = Vegetation_Indices.NDVI(AbsRef_Smoothed_WBRemoved_DrydenEM_groups)
Hyperspectral_NDVI = Hyperspectral_NDVI.rename({'Brashes_mean': 'Brashes', 'MossPatches_mean': 'Moss Patches',
                         'soiltrue_mean': 'Soil', 'vegetation_mean':'vegetation'})
Hyperspectral_NDVI = Hyperspectral_NDVI.to_frame()
Hyperspectral_NDVI = Hyperspectral_NDVI.rename(columns={0:"NDVI_Hyperspectral"})

print(Hyperspectral_NDVI)

Multispectral_NDVI = collated_convolved.loc['NDVI']
Multispectral_NDVI = Multispectral_NDVI.rename(index={'Bands_Brashes_meanSRF': 'Brashes', 'Bands_MossPatches_meanSRF': 'Moss Patches',
                         'Bands_soiltrue_meanSRF': 'Soil', 'Bands_vegetation_meanSRF':'vegetation'})
Multispectral_NDVI = Multispectral_NDVI.to_frame()
Multispectral_NDVI = Multispectral_NDVI.rename(columns={'NDVI':'NDVI_Multipectral'})
print(Multispectral_NDVI)

In [ ]:
NDVI_Comparison = pd.concat([Multispectral_NDVI, Hyperspectral_NDVI] ,axis = 1)
print(NDVI_Comparison)

With our comparison collated into one data frame, we can now visually compare the two datasets by plotting:

In [ ]:
NDVI_Comparison.plot(figsize=(9, 6), legend = True, ylim=(0,1))
xlabel("Vegetation Type")
ylabel("Absolute Reflectance")
title("Comaprison between NDVI acquired from \n a. hyperspectral data and b. hyperspectral data convolved to mutlispectral")
legend(title = "")
plt.tight_layout()
plt.savefig("NDVI_Hyperspectral_vs_Multispectral.pdf")
plt.show()

### Optional -- Cleaning the Work Directory
We have generated a lot of .csv files for processing, as a means to illustrate the step-by-step procedure involved. These .csv files are now superflous. You may keep them if you wish, in which case, do not run this cell. If not, this cell will delete the csv folder containing these files. 

In [ ]:
shutil.rmtree(csv_Dir)

### NOTE: KEEP THIS WORKBOOK OPEN SO YOU CAN USE SOME OF THE INFORMATION IN THE NEXT PRACTICAL 